# 05 — Export raw samples and plot

This notebook demonstrates:

- collecting raw samples with `store_samples=True`
- exporting to CSV
- plotting util.gpu over time (matplotlib)

If PyTorch/CUDA is available, we generate a bursty workload. Otherwise we just sleep.


In [1]:
import csv
import time

try:
    import torch
except Exception:
    torch = None

from profgpu import GpuMonitor

/cb/home/omids/ws/anaconda3/envs/profgpu/lib/python3.11/site-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
def bursty_work(duration_s=6.0):
    """Alternates GPU work and CPU sleeps to create a visible utilization pattern."""
    if torch is None or not torch.cuda.is_available():
        time.sleep(duration_s)
        return

    n = 4096
    a = torch.randn(n, n, device="cuda")
    b = torch.randn(n, n, device="cuda")
    end = time.perf_counter() + duration_s
    while time.perf_counter() < end:
        # GPU burst
        for _ in range(3):
            _ = a @ b
        # CPU gap
        time.sleep(0.15)


sync_fn = torch.cuda.synchronize if (torch is not None and torch.cuda.is_available()) else None

with GpuMonitor(interval_s=0.1, store_samples=True, sync_fn=sync_fn, strict=False) as mon:
    bursty_work(6.0)

print(mon.summary.format())

[GPU 0] NVIDIA A10G
  duration: 6.064s | samples: 58 @ 0.100s
  util.gpu: mean 10.4% | p50 8.0% | p95 17.0% | max 17.0%
  util.mem: mean 2.5%
  memory: max used 2306 MB / total 23028 MB
  power: mean 114.1 W | max 134.3 W
  temp: max 30 °C
  busy time (est): 0.632s
  util trace: ▁▁▅▄▄▄▄▄█▄▄▄█▄▄▄██▄▄█▄▄▄█▄▄▄█▅▅▄█▅▄██▅▄▇


In [3]:
# Export to CSV
with open("gpu_samples.csv", "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["t_s", "util_gpu", "util_mem", "mem_used_mb", "power_w", "temp_c"])
    for s in mon.samples:
        w.writerow([s.t_s, s.util_gpu, s.util_mem, s.mem_used_mb, s.power_w, s.temp_c])

print("Wrote gpu_samples.csv with", len(mon.samples), "rows")

Wrote gpu_samples.csv with 58 rows


In [4]:
# Plot util.gpu over time
try:
    import matplotlib.pyplot as plt
except Exception as e:
    plt = None
    print("matplotlib not available:", e)

if plt is not None:
    t = [s.t_s for s in mon.samples if s.util_gpu is not None]
    u = [s.util_gpu for s in mon.samples if s.util_gpu is not None]
    plt.figure()
    plt.plot(t, u)
    plt.xlabel("t (s)")
    plt.ylabel("util.gpu (%)")
    plt.title("GPU utilization over time")
    plt.show()

matplotlib not available: No module named 'matplotlib'
